In [1]:
import json
import os
import pandas as pd
naver_path = '../news data/articles/all_daily_articles_naver/'
daum_path = '../news data/articles/all_daily_articles_daum/'
naver_main_path = '../news data/main/main_naver/main/'
daum_main_path = '../news data/main/main_daum/'
naver_list = os.listdir(naver_path)
daum_list = os.listdir(daum_path)
naver_main_list = os.listdir(naver_main_path)
daum_main_list = os.listdir(daum_main_path)


print('NAVER FROM {} to {}'.format(naver_list[0], naver_list[-1]))
print('DAUM FROM {} to {}'.format(daum_list[0], daum_list[-1]))

print('NAVER MAIN FROM {} to {}'.format(naver_main_list[0], naver_main_list[-1]))
print('DAUM MAIN FROM {} to {}'.format(daum_main_list[0], daum_main_list[-1]))

NAVER FROM 20000202.json to 20191002.json
DAUM FROM 20060101.json to 20190726.json
NAVER MAIN FROM 2009-06-07.json to 2019-04-04.json
DAUM MAIN FROM .DS_Store to SP


In [25]:
'''
각 날짜별 main info file을 만드는 기본이 되는 코드.
각 기사 별 다음/네이버 등재 여부, 댓글 수, 메인 등재 여부, 메인 노출 시작/끝 시간 (네이버), 메인에 등재된 날짜 등을
담을 수 있는 파일을 작성.
'''
from collections import defaultdict
import csv
from tqdm import tqdm
import datetime

def best_comment(v, day, file, portal):
    if portal == 'naver':
        if not v.get('naver'):
            return None
        else:
            articles = v.get('naver')
            if len(articles) == 1: 
                return articles[0]
            else:
                if not file:
                    return articles[0]
                else:
                    result = []
                    for article in articles:
                        if file.get(article):
                            result.append((article, file[article]['comment']))
                        else:
                            result.append((article, 0))

                    sorted_by_comment = sorted(
                        result, key=lambda x: x[1], reverse=True)
                    return sorted_by_comment[0][0]
    else:
        if not v.get('daum'):
            return None
        else:
            articles = v.get('daum')
            if len(articles) == 1: 
                return articles[0]
            else:
                if not file:
                    return articles[0]
                else:
                    result = []
                    for article in articles:
                        if file.get(article):
                            result.append((article, file[article]['commentCount']))
                        else:
                            result.append((article, 0))

                    sorted_by_comment = sorted(
                        result, key=lambda x: x[1], reverse=True)
                    return sorted_by_comment[0][0]


def comment_count(aid, file, portal):
    if aid == '':
        return ''
    
    if not file:
        return ''
    if file.get(aid):
        if portal == 'naver':
            return file[aid]['comment']
        else:
            return file[aid]['commentCount']
    else:
        return ''


for day in tqdm(naver_main_list[1788:]):
    day = day[:4]+day[5:7]+day[8:10]+'.json'

    # 각 날짜 별 기사 기본 정보 파일 불러오기
    with open(naver_path + day, 'r') as f:
        naver_article = json.load(f)

    if '20060407' <= day[:8] <= '20190706':
        with open('../news data/articles/comment_count_naver/' + day, 'r') as f:
            naver_comment_json = json.load(f)
    else:
        naver_comment_json = None

    try:
        with open('../news data/articles/all_daily_articles_meta/'+day, 'r') as f:
            daum_comment_json = json.load(f)
    except:
        daum_comment_json = None

    # 다음의 경우에는 그냥 meta 파일 불러오면 끝. 파일 폴더 바꾸어주기만 하면 됨.
    with open(daum_path + day, 'r') as f:
        daum_article = json.load(f)

    nav = {}
    for k, v in daum_article.items():
        key = (v['title'], v['office'])
        if not nav.get(key):
            nav[key] = {}
            nav[key]['daum'] = []
            nav[key]['daum'].append(k)
        else:
            nav[key]['daum'].append(k)

    for k, v in naver_article.items():
        key = key = (v['title'], v['office'])
        if nav.get(key):
            if not nav[key].get('naver'):
                nav[key]['naver'] = []
                nav[key]['naver'].append(k)
            else:
                nav[key]['naver'].append(k)
        else:
            nav[key] = {}
            nav[key]['naver'] = [] 
            nav[key]['naver'].append(k)

#     nav = {k: v for k, v in nav.items() if len(v) > 1}

    nav = {k: {'naver': best_comment(v, day, naver_comment_json, 'naver'), 'daum': best_comment(
        v, day, daum_comment_json, 'daum')} for k, v in nav.items()}
    naver_ON = 0
    daum_ON = 0

    naver_file_name = day[0:4] + '-' + day[4:6] + '-' + day[6:]
    with open(naver_main_path + naver_file_name, 'r') as f:
        naver_main_info = json.load(f)
    naver_ON = 1

    daum_file_news = 'NE_' + day
    with open(daum_main_path + 'NE/' + daum_file_news, 'r') as f:
        daum_main_info = json.load(f)
    daum_file_enter = 'EN_' + day
    with open(daum_main_path + 'EN/' + daum_file_enter, 'r') as f:
        daum_main_enter = json.load(f)
    daum_main_info.extend(daum_main_enter)
    daum_file_sports = 'SP_' + day
    with open(daum_main_path + 'SP/' + daum_file_sports, 'r') as f:
        daum_main_sports = json.load(f)
    daum_main_info.extend(daum_main_sports)
    daum_ON = 1

    for title, office in list(nav.keys()):
        key = (title, office)
        nav[key]['naver_main'] = 0 if nav[key].get('naver') else ""
        nav[key]['daum_main'] = 0 if nav[key].get('daum') else ""
        nav[key]['naver_main_date'] = ""
        nav[key]['daum_main_date'] = ""
        nav[key]['naver_main_exposed_first'] = ""
        nav[key]['naver_main_exposed_last'] = ""
        if naver_ON:
            if '20140430' > day[:8]:
                for info in naver_main_info:
                    if info['title'] == title and info['office'] == office:
                        nav[key]['naver_main_date'] = naver_counter[key]
                        nav[key]['naver_main'] = 1
                        break
            else:
                if nav[key].get('naver'):
                    nav_aid = nav[key].get('naver')
                    if naver_new_main_info.get(nav_aid):
                        nav[key]['naver_main'] = 1
                        ef = datetime.datetime.strptime(
                            naver_new_main_info[nav_aid]['expose_first'], '%Y-%m-%d %H:%M')
                        el = datetime.datetime.strptime(
                            naver_new_main_info[nav_aid]['expose_last'], '%Y-%m-%d %H:%M')
                        nav[key]['naver_main_exposed_first'] = ef
                        nav[key]['naver_main_exposed_last'] = el
                        nav[key]['naver_main_date'] = '-'.join(
                            sorted(set(map(lambda x: x.strftime('%Y%m%d'),[ef, el])))
                        )
        if daum_ON:
            for info in daum_main_info:
                if info['title'] == title and info['office'] == office:
                    nav[key]['daum_main_date'] = daum_counter[key]
                    nav[key]['daum_main'] = 1
                    break

    for k, v in nav.items():
        nav[k]['naver_comment_count'] = comment_count(
            v['naver'], naver_comment_json, 'naver')
        nav[k]['daum_comment_count'] = comment_count(
            v['daum'], daum_comment_json, 'daum')

    with open('../news data/articles/20200303_duplicated/' + day[:8] + '.csv', 'w', encoding='utf-8') as f:
        wr = csv.writer(f)
        wr.writerow(['office', 'aid', 'naver_id', 'daum_id', 'naver_main',
                     'daum_main', 'naver_commentCount', 'daum_commentCount', 'naver_mainDate', 'daum_mainDate',
                    'naver_main_exposed_first','naver_main_exposed_last'])
        for k, v in nav.items():
            wr.writerow([k[1], k[0], v['naver'], v['daum'], v['naver_main'],
                         v['daum_main'], v['naver_comment_count'], v['daum_comment_count'], 
                         v['naver_main_date'], v['daum_main_date'],
                        v['naver_main_exposed_first'], v['naver_main_exposed_last']])

    f.close()




  0%|          | 0/1801 [00:00<?, ?it/s]


  0%|          | 1/1801 [00:03<1:40:02,  3.33s/it]


  0%|          | 2/1801 [00:06<1:35:37,  3.19s/it]


  0%|          | 3/1801 [00:09<1:36:41,  3.23s/it]


  0%|          | 4/1801 [00:11<1:21:52,  2.73s/it]


  0%|          | 5/1801 [00:12<1:10:59,  2.37s/it]


  0%|          | 6/1801 [00:14<1:02:53,  2.10s/it]


  0%|          | 7/1801 [00:16<1:01:59,  2.07s/it]


  0%|          | 8/1801 [00:20<1:18:40,  2.63s/it]


  0%|          | 9/1801 [00:24<1:32:29,  3.10s/it]


  1%|          | 10/1801 [00:27<1:36:42,  3.24s/it]


  1%|          | 11/1801 [00:29<1:21:24,  2.73s/it]


  1%|          | 12/1801 [00:31<1:12:31,  2.43s/it]


  1%|          | 13/1801 [00:35<1:28:02,  2.95s/it]


  1%|          | 14/1801 [00:39<1:40:40,  3.38s/it]


  1%|          | 15/1801 [00:44<1:51:33,  3.75s/it]


  1%|          | 16/1801 [00:48<1:57:50,  3.96s/it]


  1%|          | 17/1801 [00:52<1:55:23,  3.88s/it]


  1%|          | 18/1801 [00:54<1:35:44,  3.2

 14%|█▍        | 257/1801 [15:07<1:23:30,  3.24s/it]


 14%|█▍        | 258/1801 [15:12<1:33:13,  3.63s/it]


 14%|█▍        | 259/1801 [15:17<1:45:08,  4.09s/it]


 14%|█▍        | 260/1801 [15:22<1:49:16,  4.25s/it]


 14%|█▍        | 261/1801 [15:27<1:54:35,  4.46s/it]


 15%|█▍        | 262/1801 [15:31<1:50:51,  4.32s/it]


 15%|█▍        | 263/1801 [15:33<1:32:43,  3.62s/it]


 15%|█▍        | 264/1801 [15:35<1:23:10,  3.25s/it]


 15%|█▍        | 265/1801 [15:40<1:34:04,  3.67s/it]


 15%|█▍        | 266/1801 [15:45<1:43:17,  4.04s/it]


 15%|█▍        | 267/1801 [15:50<1:50:44,  4.33s/it]


 15%|█▍        | 268/1801 [15:55<1:58:11,  4.63s/it]


 15%|█▍        | 269/1801 [15:59<1:56:17,  4.55s/it]


 15%|█▍        | 270/1801 [16:01<1:35:27,  3.74s/it]


 15%|█▌        | 271/1801 [16:03<1:22:23,  3.23s/it]


 15%|█▌        | 272/1801 [16:08<1:34:57,  3.73s/it]


 15%|█▌        | 273/1801 [16:13<1:45:37,  4.15s/it]


 15%|█▌        | 274/1801 [16:19<1:54:34,  4.50s/it]


 15%|█▌   

 28%|██▊       | 513/1801 [33:57<2:13:18,  6.21s/it]


 29%|██▊       | 514/1801 [34:02<2:06:15,  5.89s/it]


 29%|██▊       | 515/1801 [34:05<1:45:19,  4.91s/it]


 29%|██▊       | 516/1801 [34:07<1:30:22,  4.22s/it]


 29%|██▊       | 517/1801 [34:09<1:16:37,  3.58s/it]


 29%|██▉       | 518/1801 [34:12<1:11:16,  3.33s/it]


 29%|██▉       | 519/1801 [34:18<1:29:29,  4.19s/it]


 29%|██▉       | 520/1801 [34:25<1:44:52,  4.91s/it]


 29%|██▉       | 521/1801 [34:31<1:53:56,  5.34s/it]


 29%|██▉       | 522/1801 [34:34<1:36:23,  4.52s/it]


 29%|██▉       | 523/1801 [34:37<1:29:04,  4.18s/it]


 29%|██▉       | 524/1801 [34:44<1:43:45,  4.88s/it]


 29%|██▉       | 525/1801 [34:50<1:54:21,  5.38s/it]


 29%|██▉       | 526/1801 [34:57<2:02:30,  5.77s/it]


 29%|██▉       | 527/1801 [35:03<2:01:55,  5.74s/it]


 29%|██▉       | 528/1801 [35:06<1:46:00,  5.00s/it]


 29%|██▉       | 529/1801 [35:08<1:29:17,  4.21s/it]


 29%|██▉       | 530/1801 [35:12<1:22:28,  3.89s/it]


 29%|██▉  

 43%|████▎     | 769/1801 [58:39<1:11:41,  4.17s/it]


 43%|████▎     | 770/1801 [58:45<1:24:23,  4.91s/it]


 43%|████▎     | 771/1801 [58:53<1:38:08,  5.72s/it]


 43%|████▎     | 772/1801 [59:00<1:45:00,  6.12s/it]


 43%|████▎     | 773/1801 [59:06<1:41:19,  5.91s/it]


 43%|████▎     | 774/1801 [59:08<1:24:42,  4.95s/it]


 43%|████▎     | 775/1801 [59:11<1:14:40,  4.37s/it]


 43%|████▎     | 776/1801 [59:17<1:24:15,  4.93s/it]


 43%|████▎     | 777/1801 [59:25<1:35:38,  5.60s/it]


 43%|████▎     | 778/1801 [59:31<1:40:47,  5.91s/it]


 43%|████▎     | 779/1801 [59:38<1:47:12,  6.29s/it]


 43%|████▎     | 780/1801 [59:44<1:44:00,  6.11s/it]


 43%|████▎     | 781/1801 [59:47<1:25:22,  5.02s/it]


 43%|████▎     | 782/1801 [59:49<1:13:30,  4.33s/it]


 43%|████▎     | 783/1801 [59:55<1:22:27,  4.86s/it]


 44%|████▎     | 784/1801 [1:00:03<1:35:06,  5.61s/it]


 44%|████▎     | 785/1801 [1:00:10<1:42:07,  6.03s/it]


 44%|████▎     | 786/1801 [1:00:17<1:48:05,  6.39s/it]


 44%

 56%|█████▋    | 1017/1801 [1:21:42<1:26:52,  6.65s/it]


 57%|█████▋    | 1018/1801 [1:21:48<1:23:04,  6.37s/it]


 57%|█████▋    | 1019/1801 [1:21:51<1:07:55,  5.21s/it]


 57%|█████▋    | 1020/1801 [1:21:54<1:00:22,  4.64s/it]


 57%|█████▋    | 1021/1801 [1:22:01<1:09:44,  5.36s/it]


 57%|█████▋    | 1022/1801 [1:22:09<1:18:58,  6.08s/it]


 57%|█████▋    | 1023/1801 [1:22:15<1:20:18,  6.19s/it]


 57%|█████▋    | 1024/1801 [1:22:23<1:26:15,  6.66s/it]


 57%|█████▋    | 1025/1801 [1:22:29<1:23:11,  6.43s/it]


 57%|█████▋    | 1026/1801 [1:22:31<1:07:50,  5.25s/it]


 57%|█████▋    | 1027/1801 [1:22:34<59:11,  4.59s/it]  


 57%|█████▋    | 1028/1801 [1:22:41<1:05:35,  5.09s/it]


 57%|█████▋    | 1029/1801 [1:22:48<1:13:10,  5.69s/it]


 57%|█████▋    | 1030/1801 [1:22:55<1:20:31,  6.27s/it]


 57%|█████▋    | 1031/1801 [1:23:02<1:22:56,  6.46s/it]


 57%|█████▋    | 1032/1801 [1:23:09<1:21:47,  6.38s/it]


 57%|█████▋    | 1033/1801 [1:23:11<1:08:19,  5.34s/it]


 57%|█████▋   

 70%|███████   | 1262/1801 [1:45:19<52:41,  5.87s/it]


 70%|███████   | 1263/1801 [1:45:25<53:30,  5.97s/it]


 70%|███████   | 1264/1801 [1:45:27<43:40,  4.88s/it]


 70%|███████   | 1265/1801 [1:45:31<40:11,  4.50s/it]


 70%|███████   | 1266/1801 [1:45:38<46:29,  5.21s/it]


 70%|███████   | 1267/1801 [1:45:46<53:29,  6.01s/it]


 70%|███████   | 1268/1801 [1:45:54<58:25,  6.58s/it]


 70%|███████   | 1269/1801 [1:46:01<1:01:13,  6.90s/it]


 71%|███████   | 1270/1801 [1:46:09<1:02:49,  7.10s/it]


 71%|███████   | 1271/1801 [1:46:12<51:00,  5.77s/it]  


 71%|███████   | 1272/1801 [1:46:15<43:34,  4.94s/it]


 71%|███████   | 1273/1801 [1:46:22<48:38,  5.53s/it]


 71%|███████   | 1274/1801 [1:46:29<53:38,  6.11s/it]


 71%|███████   | 1275/1801 [1:46:38<1:02:12,  7.10s/it]


 71%|███████   | 1276/1801 [1:46:47<1:06:14,  7.57s/it]


 71%|███████   | 1277/1801 [1:46:54<1:04:34,  7.39s/it]


 71%|███████   | 1278/1801 [1:46:57<52:57,  6.07s/it]  


 71%|███████   | 1279/1801 [1:47:0

 84%|████████▍ | 1513/1801 [2:10:02<31:46,  6.62s/it]


 84%|████████▍ | 1514/1801 [2:10:09<32:42,  6.84s/it]


 84%|████████▍ | 1515/1801 [2:10:15<32:09,  6.75s/it]


 84%|████████▍ | 1516/1801 [2:10:18<26:43,  5.62s/it]


 84%|████████▍ | 1517/1801 [2:10:22<23:59,  5.07s/it]


 84%|████████▍ | 1518/1801 [2:10:28<25:40,  5.44s/it]


 84%|████████▍ | 1519/1801 [2:10:36<29:02,  6.18s/it]


 84%|████████▍ | 1520/1801 [2:10:44<31:17,  6.68s/it]


 84%|████████▍ | 1521/1801 [2:10:52<33:04,  7.09s/it]


 85%|████████▍ | 1522/1801 [2:10:59<31:54,  6.86s/it]


 85%|████████▍ | 1523/1801 [2:11:01<25:21,  5.47s/it]


 85%|████████▍ | 1524/1801 [2:11:04<21:44,  4.71s/it]


 85%|████████▍ | 1525/1801 [2:11:11<24:37,  5.35s/it]


 85%|████████▍ | 1526/1801 [2:11:18<27:49,  6.07s/it]


 85%|████████▍ | 1527/1801 [2:11:26<30:05,  6.59s/it]


 85%|████████▍ | 1528/1801 [2:11:34<31:16,  6.87s/it]


 85%|████████▍ | 1529/1801 [2:11:40<30:03,  6.63s/it]


 85%|████████▍ | 1530/1801 [2:11:42<24:26,  5.41

 98%|█████████▊| 1765/1801 [2:34:59<04:13,  7.05s/it]


 98%|█████████▊| 1766/1801 [2:35:07<04:20,  7.43s/it]


 98%|█████████▊| 1767/1801 [2:35:11<03:33,  6.29s/it]


 98%|█████████▊| 1768/1801 [2:35:14<02:51,  5.20s/it]


 98%|█████████▊| 1769/1801 [2:35:17<02:28,  4.64s/it]


 98%|█████████▊| 1770/1801 [2:35:25<02:51,  5.53s/it]


 98%|█████████▊| 1771/1801 [2:35:32<03:05,  6.17s/it]


 98%|█████████▊| 1772/1801 [2:35:40<03:15,  6.74s/it]


 98%|█████████▊| 1773/1801 [2:35:48<03:19,  7.12s/it]


 99%|█████████▊| 1774/1801 [2:35:55<03:10,  7.05s/it]


 99%|█████████▊| 1775/1801 [2:35:58<02:29,  5.74s/it]


 99%|█████████▊| 1776/1801 [2:36:01<02:01,  4.85s/it]


 99%|█████████▊| 1777/1801 [2:36:09<02:18,  5.77s/it]


 99%|█████████▊| 1778/1801 [2:36:17<02:29,  6.50s/it]


 99%|█████████▉| 1779/1801 [2:36:25<02:34,  7.01s/it]


 99%|█████████▉| 1780/1801 [2:36:34<02:40,  7.65s/it]


 99%|█████████▉| 1781/1801 [2:36:41<02:26,  7.33s/it]


 99%|█████████▉| 1782/1801 [2:36:43<01:52,  5.94

In [26]:
# 결과 확인.
duplicated_path = '../news data/articles/20200303_duplicated/'
idx = naver_main_list.index('2014-04-30.json')
target_list = list(map(lambda x: x[:4]+x[5:7]+x[8:10]+'.csv', naver_main_list[idx:]))
main_list = naver_main_list[idx:]

df = pd.read_csv(duplicated_path + '20090608.csv', dtype = object, index_col = False)
df.loc[(df.naver_main == '1') & (df.daum_main == '1')]

,office,aid,naver_id,daum_id,naver_main,daum_main,naver_commentCount,daum_commentCount,naver_mainDate,daum_mainDate,naver_main_exposed_first,naver_main_exposed_last
395,SBS,중국산 저가 안경테 일본산 둔갑 무려 9배 폭리,"055,0000160858",20090608212111240,1,1,5,84,"20090608,20090609",20090608,NaN,NaN
3117,연합뉴스,日 아키하바라 묻지마 살상극 1년,"001,0002704466",20090608165705498,1,1,11,73,20090608,20090608,NaN,NaN
3523,스타뉴스,"'무도' 김태호PD ""길은 제8의 멤버..교체는 없다""","108,0001986366",20090608162210105,1,1,121,320,"20090608,20090609",20090608,NaN,NaN
3636,연합뉴스,"한, 비정규직법 '2년조항' 유예 잠정확정(종합)","001,0002704348",20090608161309353,1,1,30,144,"20090608,20090609",20090608,NaN,NaN
3653,연합뉴스,"靑 ""4대강 살리기, 대운하와 다르다""","001,0002704342",20090608161209506,1,1,171,1619,20090608,20090608,NaN,NaN
4264,연합뉴스,"한, 비정규직법 '2년조항' 유예 잠정확정-1,2","001,0002704165",20090608152806059,1,1,6,107,20090608,20090608,NaN,NaN
4361,머니투데이,"4대강살리기, 대운하 등 2大 논란 여전","008,0002158793",20090608152203940,1,1,38,400,20090608,20090608,NaN,NaN
5723,연합뉴스,경기선행 10대지표 7년만에 '올 플러스',"001,0002703038",20090608061403066,1,1,23,70,20090608,20090608,NaN,NaN
11734,연합뉴스,돈 없다..술 지출 환란후 첫 감소,"001,0002703035",20090608061204697,1,1,1,15,20090608,20090608,NaN,NaN
11735,연합뉴스,"""휴대전화 도금케이스, 통화품질 저하시켜""","001,0002703034",20090608061203299,1,1,6,5,20090608,20090608,NaN,NaN
